#### Experiments to be sorted later

COMBINATION ID VERY EARLY - INSERT IT FOR LATER EVALUATION

What I need to do:

-Parameter tuning of RF



Data cleaning:
-average double entries
-use maccskeys for a RF

New dataset:
-fix variability above 100 down to 100
-

In [1]:
# temporary working here to deal with double values and insert it later as new section before section joining

In [1]:
import numpy as np
import pandas as pd
import _pickle as pkl

#### 2. Joining

In [19]:
with open('./intermediate-pickles-scoring2/dataWithSource.pkl', 'rb') as inp:
    data1 = pkl.load(inp)
with open('./intermediate-pickles-scoring2/data2.pkl', 'rb') as inp:
    data2 = pkl.load(inp)

In [20]:
data2 = data2[data2["HSA"]<84.4]

In [21]:
# double Data
dataReversed1 = data1
dataReversed2 = data2

dataReversed1 = dataReversed1.rename(columns={"Drug1":"Drug2","Drug2":"Drug1"})
dataReversed2 = dataReversed2.rename(columns={"Drug1":"Drug2","Drug2":"Drug1"})

dataDouble1 = data1.append(dataReversed1)
dataDouble2 = data2.append(dataReversed2)

dataDouble1.reset_index(inplace=True,drop=True)
dataDouble2.reset_index(inplace=True,drop=True)

print(len(dataDouble1)," records in first frame")
print(len(dataDouble2)," records in second frame")
dataDouble1.head(2)

492814  records in first frame
811376  records in second frame


,drug_drug2,ID,Drug1,Drug2,Cell line,ZIP,Bliss,Loewe,index,HSA,source
0,5-fuabt-888A2058,1,5-fu,abt-888,A2058,2.80125,9.4975,0.490625,284469,8.7775,ONEIL
1,5-fuazd1775A2058,5,5-fu,azd1775,A2058,9.81750,11.2075,5.837500,29611,14.0975,ONEIL


Drug Feature import part

In [23]:
drug_feats = pd.read_csv("./output/all_drugs_with_smiles.csv")
drug_feats["mol-object"]=[Chem.MolFromSmiles(mol) for mol in drug_feats["smilesString"]]

maccsFrame = pd.DataFrame(drug_feats["drugName"])
maccsFrame["MACCSk"] = [np.array(Chem.MACCSkeys.GenMACCSKeys(mol)) for mol in chemDescr["mol-object"]]

joinMaccs = pd.DataFrame([])
for i in range(len(maccsFrame)):
    joinMaccs = joinMaccs.append(pd.Series(maccsFrame.iloc[i,1]),ignore_index=True)

drugMACCS = maccsFrame.join(joinMaccs,how="left").drop(columns="MACCSk")

In [24]:
drug_feats.set_index("drugName",inplace=True)
drug_feats.index = drug_feats.index.str.lower()

d1_drugs = drug_feats.add_prefix("D1 ")
d2_drugs = drug_feats.add_prefix("D2 ")

dataDouble_drug1_feats1 = dataDouble1.join(d1_drugs,on="Drug1",how="left")
dataDouble_drug1_feats2 = dataDouble2.join(d1_drugs,on="Drug1",how="left")

dataDouble_all_feats1 = dataDouble_drug1_feats1.join(d2_drugs,on="Drug2",how="left")
dataDouble_all_feats2 = dataDouble_drug1_feats2.join(d2_drugs,on="Drug2",how="left")

Cell Line Info from Ammar

In [26]:
# Ammars output files
cellLine_info = pd.read_csv('output/cell_lines_enriched.csv', sep=',', dtype='string')
drug_info = pd.read_csv('output/all_drugs_with_smiles.csv', sep=',', dtype='string')

cat_ohe = pd.get_dummies(cellLine_info.category, prefix='cat')
cellLine_info_full = pd.concat([cellLine_info,cat_ohe], axis=1)

gender_ohe = pd.get_dummies(cellLine_info.gender, prefix='gender')

cellLine_info_full['age'] = cellLine_info_full['age'].str.replace('Y','').replace('Age unspecified','0')
cellLine_info_full['age'] = cellLine_info_full['age'].astype('int')

cellLine_info_full = cellLine_info_full.drop('cosmicId', 1)
cellLine_info_full = cellLine_info_full.drop('category', 1)
cellLine_info_full = cellLine_info_full.drop('gender', 1)

In [27]:
finalFrame1 = dataDouble_all_feats1.join(cellLine_info_full.set_index("cellName"),on="Cell line", how="left")#.dropna()
finalFrame2 = dataDouble_all_feats2.join(cellLine_info_full.set_index("cellName"),on="Cell line", how="left")#.dropna()

In [29]:
with open('./intermediate-pickles-scoring2/finalFrame1.pkl', 'wb') as outp:
    pkl.dump(finalFrame1, outp)
with open('./intermediate-pickles-scoring2/finalFrame2.pkl', 'wb') as outp:
    pkl.dump(finalFrame2, outp)

#### 3. Combination ID?

In [38]:
with open('./intermediate-pickles-scoring2/finalFrame1.pkl', 'rb') as inp:
    data1 = pkl.load(inp)
with open('./intermediate-pickles-scoring2/finalFrame2.pkl', 'rb') as inp:
    data2 = pkl.load(inp)

In [39]:
data1 = data1.dropna() #loose here 10-20% of data
data2 = data2.dropna()
data1.reset_index(inplace=True,drop=True)
data2.reset_index(inplace=True,drop=True)
print("Data1 has records ",len(data1))
print("Data2 has records ",len(data2))
data1.head(2)

Data1 has records  457614
Data2 has records  603376


,drug_drug2,ID,Drug1,Drug2,Cell line,ZIP,Bliss,Loewe,index,HSA,...,D2 EState_VSA2,D2 EState_VSA3,D2 VSA_EState1,D2 VSA_EState2,D2 VSA_EState3,age,cat_Cancer cell line,cat_Embryonic stem cell,cat_Hybridoma,cat_Transformed cell line
0,5-fuabt-888A2058,1,5-fu,abt-888,A2058,2.80125,9.4975,0.490625,284469,8.7775,...,5.538925,11.080152,0.000000,19.238682,3.448989,43.0,1.0,0.0,0.0,0.0
1,5-fuazd1775A2058,5,5-fu,azd1775,A2058,9.81750,11.2075,5.837500,29611,14.0975,...,12.104023,28.493531,3.162078,31.601287,14.099736,43.0,1.0,0.0,0.0,0.0


#### 4. Making Predictions

In [50]:
Y1 = data1.iloc[:,9]
X1 = data1.iloc[:,11:]
X1.drop(["D1 Ipc","D2 Ipc"],axis=1,inplace=True)

Y2 = data2.iloc[:,9]
X2 = data2.iloc[:,10:]
X2.drop(["D1 Ipc","D2 Ipc"],axis=1,inplace=True)

from sklearn.model_selection import train_test_split
X_train1,x_test1,Y_train1,y_test1 = train_test_split(X1,Y1,test_size=0.3,stratify=Y1) #consider stratifying on combination
X_train2,x_test2,Y_train2,y_test2 = train_test_split(X2,Y2,test_size=0.3,stratify=Y2)

In [54]:
with open('./intermediate-pickles-scoring2/trainTestSplit/X_train1.pkl', 'wb') as outp:
    pkl.dump(X_train1, outp)
with open('./intermediate-pickles-scoring2/trainTestSplit/X_train2.pkl', 'wb') as outp:
    pkl.dump(X_train2, outp)
with open('./intermediate-pickles-scoring2/trainTestSplit/x_test1.pkl', 'wb') as outp:
    pkl.dump(x_test1, outp)
with open('./intermediate-pickles-scoring2/trainTestSplit/x_test2.pkl', 'wb') as outp:
    pkl.dump(x_test2, outp)

with open('./intermediate-pickles-scoring2/trainTestSplit/Y_train1.pkl', 'wb') as outp:
    pkl.dump(Y_train1, outp)
with open('./intermediate-pickles-scoring2/trainTestSplit/Y_train2.pkl', 'wb') as outp:
    pkl.dump(Y_train2, outp)
with open('./intermediate-pickles-scoring2/trainTestSplit/y_test1.pkl', 'wb') as outp:
    pkl.dump(y_test1, outp)
with open('./intermediate-pickles-scoring2/trainTestSplit/y_test2.pkl', 'wb') as outp:
    pkl.dump(y_test2, outp)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor 

regressor1 = RandomForestRegressor(max_depth=4, random_state=0, n_estimators=100, verbose=True)
regressor1.fit(X_train1,Y_train1)

predTrain1=regressor1.predict(X_train1)
predTest1=regressor1.predict(x_test1)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.7s finished


In [60]:
regressor2 = RandomForestRegressor(max_depth=4, random_state=0, n_estimators=100, verbose=True)
regressor2.fit(X_train2,Y_train2)

predTrain2=regressor2.predict(X_train2)
predTest2=regressor2.predict(x_test2)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 14.1min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    1.9s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.8s finished


In [61]:
from sklearn.dummy import DummyRegressor
dummyRegressor1 = DummyRegressor()
dummyRegressor1.fit(X_train1,Y_train1)

predTrainDumm1=dummyRegressor1.predict(X_train1)
predTestDumm1=dummyRegressor1.predict(x_test1)

dummyRegressor2 = DummyRegressor()
dummyRegressor2.fit(X_train2,Y_train2)

predTrainDumm2=dummyRegressor2.predict(X_train2)
predTestDumm2=dummyRegressor2.predict(x_test2)

#### Scoring (Mean Absolute/Squared Error)

In [69]:
print("## The performances are:")
print("\nSmall Dataset:")
print("On Training Data: ",np.mean(np.abs(predTrain1-Y_train1)))
print("On Testing  Data: ",np.mean(np.abs(predTest1-y_test1)))
print("Dummy Training  : ",np.mean(np.abs(predTrainDumm1-Y_train1)))
print("Dummy Testing   : ",np.mean(np.abs(predTestDumm1-y_test1)))
print("\nBig Dataset:")
print("On Training Data: ",np.mean(np.abs(predTrain2-Y_train2)))
print("On Testing  Data: ",np.mean(np.abs(predTest2-y_test2)))
print("Dummy Training  : ",np.mean(np.abs(predTrainDumm2-Y_train2)))
print("Dummy Testing   : ",np.mean(np.abs(predTestDumm2-y_test2)))

## The performances are:

Small Dataset:
On Training Data:  3.8512295736362114
On Testing  Data:  3.856709127409338
Dummy Training  :  4.019826881637109
Dummy Testing   :  4.020777303474963

Big Dataset:
On Training Data:  4.023602782805237
On Testing  Data:  4.038416459991488
Dummy Training  :  4.17312462139542
Dummy Testing   :  4.188929710733737


In [70]:
print("## The performances are:")
print("\nSmall Dataset:")
print("On Training Data: ",np.mean(np.square(predTrain1-Y_train1)))
print("On Testing  Data: ",np.mean(np.square(predTest1-y_test1)))
print("Dummy Training  : ",np.mean(np.square(predTrainDumm1-Y_train1)))
print("Dummy Testing   : ",np.mean(np.square(predTestDumm1-y_test1)))
print("\nBig Dataset:")
print("On Training Data: ",np.mean(np.square(predTrain2-Y_train2)))
print("On Testing  Data: ",np.mean(np.square(predTest2-y_test2)))
print("Dummy Training  : ",np.mean(np.square(predTrainDumm2-Y_train2)))
print("Dummy Testing   : ",np.mean(np.square(predTestDumm2-y_test2)))

## The performances are:

Small Dataset:
On Training Data:  32.484186707350744
On Testing  Data:  32.84695566451583
Dummy Training  :  36.058513836791256
Dummy Testing   :  36.36555926960286

Big Dataset:
On Training Data:  34.73133734048182
On Testing  Data:  35.346908681521796
Dummy Training  :  38.12436906145257
Dummy Testing   :  38.80865347982537


In [71]:
#could call scoring function as well

Next steps:

Use Keys & Cell Line Profiles